### Import Dependencies

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

from config import conn_string

**get all files for loading**

In [2]:
# path
station_path = os.path.join('.', "station_info.csv")
weather_path = os.path.join('.', "historical_weather.csv")

#bikes
toronto_bikes = os.path.join("Resources/1_toronto_canada", "toronto_bikes.csv")
vancouver_bikes = os.path.join("Resources/2_vancouver_canada", "vancouver_bikes.csv")
boston_bikes = os.path.join("Resources/3_boston_us", "boston_bikes.csv")
nyc_bikes = os.path.join("Resources/4_NewYork_US", "nyc_bikes.csv")

# membertypes
toronto_users = os.path.join("Resources/1_toronto_canada", "toronto_membertypes.csv")
vancouver_users = os.path.join("Resources/2_vancouver_canada", "vancouver_membertypes.csv")
boston_users = os.path.join("Resources/3_boston_us", "boston_membertypes.csv")
nyc_users = os.path.join("Resources/4_NewYork_US", "nyc_membertypes.csv")

# ridership
toronto_ = os.path.join("Resources/1_toronto_canada", "toronto_bikeshare.csv")
vancouver_ = os.path.join("Resources/2_vancouver_canada", "vancouver_bikeshare.csv")
boston_ = os.path.join("Resources/3_boston_us", "boston_bikeshare.csv")
nyc_ = os.path.join("Resources/4_NewYork_US", "nyc_bikeshare.csv")


In [3]:
# read files
stns = pd.read_csv(station_path, encoding="utf-8")
weather = pd.read_csv(weather_path, encoding="utf-8")

# bikes
tor_bikes = pd.read_csv(toronto_bikes, encoding="utf-8")
van_bikes = pd.read_csv(vancouver_bikes, encoding="utf-8")
bos_bikes = pd.read_csv(boston_bikes, encoding="utf-8")
nyc_bikes = pd.read_csv(nyc_bikes, encoding="utf-8")

# member_types
tor_users = pd.read_csv(toronto_users, encoding="utf-8")
van_users = pd.read_csv(vancouver_users, encoding="utf-8")
bos_users = pd.read_csv(boston_users, encoding="utf-8")
nyc_users = pd.read_csv(nyc_users, encoding="utf-8")

# pricing index


# ridership
toronto = pd.read_csv(toronto_, encoding="utf-8")
vancouver = pd.read_csv(vancouver_, encoding="utf-8")
boston = pd.read_csv(boston_, encoding="utf-8")
newyork = pd.read_csv(nyc_, encoding="utf-8")


**Locations**

In [4]:
# create locations table
locations = [{"location_id": 1,"city":"Toronto", "country":"Canada" }, 
             {"location_id": 2,"city":"Vancouver", "country":"Canada"},
             {"location_id": 3,"city":"Boston", "country":"USA"},
             {"location_id": 4,"city":"New York", "country":"USA"}
            ]

df = pd.DataFrame(locations)
df.set_index("location_id", inplace=True)
df


,city,country
location_id,,
1,Toronto,Canada
2,Vancouver,Canada
3,Boston,USA
4,New York,USA


**Stations**

In [5]:
stns.head()

,station_id,station_name,latitude,longitude,location_id
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,1
1,7001,Lower Jarvis St / The Esplanade,43.647830,-79.370698,1
2,7002,St. George St / Bloor St W,43.667333,-79.399429,1
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761,1
4,7004,University Ave / Elm St,43.656518,-79.389099,1


**Historical Weather**

In [6]:
weather.head()

,location_id,forecast_date,maxtempc,humidity,total_precip,avg_cloudcover,avg_windspeed
0,1,2019-01-01,1,77.250,3.6,50.625,16.875
1,1,2019-01-02,-5,69.000,0.2,41.750,8.625
2,1,2019-01-03,-1,68.500,1.3,68.250,17.750
3,1,2019-01-04,2,77.250,0.0,7.625,18.125
4,1,2019-01-05,3,81.125,0.0,23.750,11.250


**Bikes**

In [7]:
tor_bikes.head()

,bike_id,location_id
0,1296,1
1,2947,1
2,2293,1
3,283,1
4,1799,1


In [8]:
van_bikes.head()

,bike_id,location_id
0,388.0,2
1,1880.0,2
2,779.0,2
3,2123.0,2
4,444.0,2


In [9]:
bos_bikes.head()

,bike_id,location_id
0,3689,3
1,4142,3
2,1628,3
3,2969,3
4,3469,3


In [10]:
nyc_bikes.head()

,bike_id,location_id
0,15839,4
1,32723,4
2,27451,4
3,21579,4
4,35379,4


**Member Types**

In [11]:
tor_users.head()

,member_type,location_id
0,Annual Member,1
1,Casual Member,1


In [12]:
van_users.head()

,member_type,location_id
0,365 Standard,2
1,365 Plus,2
2,Vancity Community Pass,2
3,365 Day Pass Plus SALE,2
4,365 Day Founding Standard,2


In [13]:
bos_users.head()

,member_type,location_id
0,Subscriber,3
1,Customer,3


In [14]:
nyc_users.head()

,member_type,location_id
0,Subscriber,4
1,Customer,4


**Pricing Index**

**Ridership**

In [15]:
# Toronto
toronto.head()

,trip_duration,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,location_id
0,1547,01/01/2019 00:08,01/01/2019 00:33,7021,7233,1296,Annual Member,1
1,1112,01/01/2019 00:10,01/01/2019 00:29,7160,7051,2947,Annual Member,1
2,589,01/01/2019 00:15,01/01/2019 00:25,7055,7013,2293,Annual Member,1
3,259,01/01/2019 00:16,01/01/2019 00:20,7012,7235,283,Annual Member,1
4,281,01/01/2019 00:19,01/01/2019 00:24,7041,7257,1799,Annual Member,1


In [16]:
# Vancouver
vancouver.head()

,trip_duration,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,location_id
0,177,2/1/2019 0:00,2/1/2019 0:00,198,298,388.0,365 Standard,2
1,221,2/1/2019 0:00,2/1/2019 0:00,193,173,1880.0,365 Plus,2
2,3272,2/1/2019 0:00,2/1/2019 0:00,63,143,779.0,Vancity Community Pass,2
3,157,2/1/2019 0:00,2/1/2019 0:00,64,78,2123.0,365 Day Pass Plus SALE,2
4,516,1/31/2019 23:00,2/1/2019 0:00,60,196,444.0,365 Day Pass Plus SALE,2


In [17]:
# Boston
boston.head()

,trip_duration,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,location_id
0,371,2019-01-01 00:09:13.7980,2019-01-01 00:15:25.3360,80,179,3689,Subscriber,3
1,264,2019-01-01 00:33:56.1820,2019-01-01 00:38:20.8800,117,189,4142,Subscriber,3
2,458,2019-01-01 00:41:54.6000,2019-01-01 00:49:33.2730,68,96,1628,Subscriber,3
3,364,2019-01-01 00:43:32.5710,2019-01-01 00:49:37.4260,89,334,2969,Subscriber,3
4,681,2019-01-01 00:49:56.4640,2019-01-01 01:01:17.7010,73,367,3469,Subscriber,3


In [18]:
# New York
newyork.head()

,trip_duration,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,location_id
0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,3283.0,15839,Subscriber,4
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,518.0,32723,Subscriber,4
2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,3154.0,27451,Subscriber,4
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,3709.0,21579,Subscriber,4
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,503.0,35379,Subscriber,4


#### Load data

In [19]:
# connect to database
cnx = f'postgresql://{conn_string}'
engine = create_engine(cnx)

In [20]:
# confirm tables
engine.table_names()

['locations',
 'membertypes',
 'stations',
 'bikes',
 'ridership',
 'pricing',
 'historicalweather']

In [ ]:
# load locations into database
df.to_sql(name='locations', con=engine, if_exists='append', index=True)

In [ ]:
# load stations into database
stns.to_sql(name='stations', con=engine, if_exists='append', index=False)


In [ ]:
# load historical weather into database
weather.to_sql(name='historicalweather', con=engine, if_exists='append', index=False)

In [ ]:
# load toronto bikes into database
tor_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)

In [ ]:
# load vancouver bikes into database
van_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)

In [ ]:
# load boston bikes into database
bos_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)

In [ ]:
# load new york bikes into database
nyc_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)

In [ ]:
# load toronto member types
tor_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)

In [ ]:
# load vancouver member types
van_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)

In [ ]:
# load boston member types
bos_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)

In [ ]:
# load new york member types
nyc_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)

**Load Ridership**

In [21]:
# check reference to stations table
# create session
session = Session(bind=engine)
# declare a base using automap_base
Base = automap_base()
# use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
Base.classes.keys()
Stations = Base.classes.stations

***Verify that all station_ids used in bikeshare exists in Stations table***

In [22]:
# get all stations
all_stations = session.query(Stations.station_id).all()
all_stns_df = pd.DataFrame(all_stations)
all_stns_df

,station_id
0,7000
1,7001
2,7002
3,7003
4,7004
...,...
2396,4248
2397,4249
2398,4250
2399,4252


***Toronto***

In [23]:
# check for Toronto records with start_station_id not in stations table
missing_start = toronto[-toronto["start_station_id"].isin(all_stns_df["station_id"])]
missing_start = missing_start.rename(columns={"start_station_id":"station_id"})

In [24]:
missing_start

,trip_duration,start_date,end_date,station_id,end_station_id,bike_id,member_type,location_id


In [25]:
# check for Toronto records with end_station_id not in stations table
missing_end = toronto[-toronto["end_station_id"].isin(all_stns_df["station_id"])]
missing_end = missing_end.rename(columns={"start_station_id":"station_id"})


In [26]:
missing_end

,trip_duration,start_date,end_date,station_id,end_station_id,bike_id,member_type,location_id


*Load rows with station_id existing in stations table*

In [27]:
toronto = toronto[toronto["start_station_id"].isin(all_stns_df["station_id"])]
toronto = toronto[toronto["end_station_id"].isin(all_stns_df["station_id"])]

In [28]:
toronto

,trip_duration,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,location_id
0,1547,01/01/2019 00:08,01/01/2019 00:33,7021,7233,1296,Annual Member,1
1,1112,01/01/2019 00:10,01/01/2019 00:29,7160,7051,2947,Annual Member,1
2,589,01/01/2019 00:15,01/01/2019 00:25,7055,7013,2293,Annual Member,1
3,259,01/01/2019 00:16,01/01/2019 00:20,7012,7235,283,Annual Member,1
4,281,01/01/2019 00:19,01/01/2019 00:24,7041,7257,1799,Annual Member,1
...,...,...,...,...,...,...,...,...
99995,465,02/27/2019 07:03,02/27/2019 07:11,7320,7323,3729,Annual Member,1
99996,360,02/27/2019 07:04,02/27/2019 07:10,7143,7252,3641,Annual Member,1
99997,447,02/27/2019 07:05,02/27/2019 07:12,7054,7033,1091,Annual Member,1
99998,407,02/27/2019 07:05,02/27/2019 07:12,7031,7129,1028,Annual Member,1


In [30]:
# get all Toronto stations
stns_toronto = session.query(Stations.station_id).filter(Stations.location_id==1).all()
stns_toronto = pd.DataFrame(stns_toronto)
stns_toronto = stns_toronto.rename(columns={"station_id": "start_station_id"})
stns_toronto

,start_station_id
0,7000
1,7001
2,7002
3,7003
4,7004
...,...
602,7660
603,7662
604,7663
605,7664


In [31]:
# remove Toronto records with start_station_id not in stations table
tor_df = pd.merge(toronto, stns_toronto, how='inner', on='start_station_id')

In [34]:
tor_df

,trip_duration,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,location_id
0,1547,01/01/2019 00:08,01/01/2019 00:33,7021,7233,1296,Annual Member,1
1,453,01/01/2019 10:35,01/01/2019 10:42,7021,7253,1118,Annual Member,1
2,632,01/01/2019 14:13,01/01/2019 14:24,7021,7323,111,Annual Member,1
3,234,01/01/2019 19:05,01/01/2019 19:09,7021,7052,942,Annual Member,1
4,843,01/02/2019 09:14,01/02/2019 09:28,7021,7036,1814,Annual Member,1
...,...,...,...,...,...,...,...,...
99995,1255,02/02/2019 14:24,02/02/2019 14:45,7345,7076,3156,Casual Member,1
99996,1240,02/02/2019 14:24,02/02/2019 14:45,7345,7076,1233,Casual Member,1
99997,1226,02/03/2019 15:23,02/03/2019 15:43,7345,7108,2974,Annual Member,1
99998,1515,02/04/2019 16:06,02/04/2019 16:31,7345,7313,851,Annual Member,1


In [32]:
# rename Toronto station column to check if end_station_id exists in stations table
stns_toronto = stns_toronto.rename(columns={"start_station_id": "end_station_id"})
stns_toronto

,end_station_id
0,7000
1,7001
2,7002
3,7003
4,7004
...,...
602,7660
603,7662
604,7663
605,7664


In [36]:
#remove Toronto records with end_station_id not in stations table
toronto = pd.merge(tor_df, stns_toronto, how='inner', on='end_station_id')

In [37]:
toronto

,trip_duration,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,location_id
0,1547,01/01/2019 00:08,01/01/2019 00:33,7021,7233,1296,Annual Member,1
1,1243,01/03/2019 17:13,01/03/2019 17:34,7021,7233,3168,Annual Member,1
2,1112,01/04/2019 21:42,01/04/2019 22:00,7021,7233,3512,Annual Member,1
3,1156,01/05/2019 23:21,01/05/2019 23:40,7021,7233,212,Annual Member,1
4,1068,01/06/2019 21:42,01/06/2019 22:00,7021,7233,3279,Annual Member,1
...,...,...,...,...,...,...,...,...
99995,1074,01/02/2019 11:23,01/02/2019 11:41,7087,7315,2609,Annual Member,1
99996,6967,02/02/2019 12:26,02/02/2019 14:23,7344,7345,795,Casual Member,1
99997,6964,02/02/2019 12:27,02/02/2019 14:23,7344,7345,1004,Casual Member,1
99998,1505,02/03/2019 14:57,02/03/2019 15:22,7344,7345,2974,Annual Member,1


In [ ]:
# Load Toronto bikeshare data to ridership table
toronto.to_sql(name='ridership', con=engine, if_exists='append', index=False, index_label='id')

***Vancouver***

In [38]:
# check for Vancouver records with start_station_id missing in stations table
missing_start = vancouver[-vancouver["start_station_id"].isin(all_stns_df["station_id"])]
missing_start = missing_start.rename(columns={"start_station_id":"station_id"})


In [39]:
missing_start

,trip_duration,start_date,end_date,station_id,end_station_id,bike_id,member_type,location_id
8712,28,1/25/2019 18:00,1/25/2019 18:00,990,990,1306.0,VIP,2
9630,170,1/25/2019 9:00,1/25/2019 9:00,990,225,1412.0,VIP,2
11715,30,1/23/2019 17:00,1/23/2019 17:00,991,990,681.0,VIP,2
12212,1600,1/23/2019 11:00,1/23/2019 11:00,990,54,412.0,VIP,2
12238,34,1/23/2019 10:00,1/23/2019 10:00,990,990,412.0,VIP,2
12249,26,1/23/2019 10:00,1/23/2019 10:00,991,991,412.0,VIP,2
12273,642,1/23/2019 10:00,1/23/2019 10:00,990,6,843.0,365 Day Pass Plus SALE,2
14000,5510,1/21/2019 14:00,1/21/2019 16:00,990,72,1775.0,365 Plus,2
14005,5755,1/21/2019 14:00,1/21/2019 16:00,990,72,1127.0,365 Plus,2
15203,768,1/20/2019 15:00,1/20/2019 16:00,990,254,504.0,VIP,2


In [40]:
# check for Vancouver records with end_station_id missing in stations table
missing_end = vancouver[-vancouver["end_station_id"].isin(all_stns_df["station_id"])]
missing_end = missing_end.rename(columns={"start_station_id":"station_id"})


In [41]:
missing_end

,trip_duration,start_date,end_date,station_id,end_station_id,bike_id,member_type,location_id
1215,1035,1/31/2019 8:00,1/31/2019 8:00,107,990,1542.0,VIP,2
4437,662,1/29/2019 6:00,1/29/2019 6:00,242,990,235.0,VIP,2
5764,885,1/28/2019 8:00,1/28/2019 9:00,15,990,824.0,VIP,2
5805,4839,1/28/2019 8:00,1/28/2019 9:00,254,991,1623.0,VIP,2
5811,1208,1/28/2019 8:00,1/28/2019 8:00,206,990,1133.0,VIP,2
6544,706,1/27/2019 14:00,1/27/2019 14:00,150,990,1034.0,VIP,2
8712,28,1/25/2019 18:00,1/25/2019 18:00,990,990,1306.0,VIP,2
9575,186,1/25/2019 10:00,1/25/2019 10:00,225,990,1412.0,VIP,2
11715,30,1/23/2019 17:00,1/23/2019 17:00,991,990,681.0,VIP,2
12238,34,1/23/2019 10:00,1/23/2019 10:00,990,990,412.0,VIP,2


In [42]:
# check Vancouver records with start_station_id and end_station_id existing in stations table
vancouver = vancouver[vancouver["start_station_id"].isin(all_stns_df["station_id"])]
vancouver = vancouver[vancouver["end_station_id"].isin(all_stns_df["station_id"])]

In [43]:
vancouver

,trip_duration,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,location_id
0,177,2/1/2019 0:00,2/1/2019 0:00,198,298,388.0,365 Standard,2
1,221,2/1/2019 0:00,2/1/2019 0:00,193,173,1880.0,365 Plus,2
2,3272,2/1/2019 0:00,2/1/2019 0:00,63,143,779.0,Vancity Community Pass,2
3,157,2/1/2019 0:00,2/1/2019 0:00,64,78,2123.0,365 Day Pass Plus SALE,2
4,516,1/31/2019 23:00,2/1/2019 0:00,60,196,444.0,365 Day Pass Plus SALE,2
...,...,...,...,...,...,...,...,...
36740,1382,1/1/2019 0:00,1/1/2019 1:00,105,84,511.0,24 Hour,2
36741,1009,1/1/2019 0:00,1/1/2019 0:00,88,218,723.0,365 Corporate Standard,2
36742,1144,1/1/2019 0:00,1/1/2019 1:00,139,167,938.0,365 Day Founding Plus,2
36743,1105,1/1/2019 0:00,1/1/2019 0:00,88,218,1450.0,365 Day Founding Plus,2


In [45]:
# get all Vancouver stations
stns_vancouver = session.query(Stations.station_id).filter(Stations.location_id==2).all()
stns_vancouver = pd.DataFrame(stns_vancouver)
stns_vancouver = stns_vancouver.rename(columns={"station_id": "start_station_id"})
stns_vancouver

,start_station_id
0,1
1,2
2,4
3,5
4,6
...,...
197,285
198,287
199,297
200,298


In [46]:
# remove Vancouver records with start_station_id not in stations table
vancouver_df = pd.merge(vancouver, stns_vancouver, how='inner', on='start_station_id')

In [48]:
# rename Vancouver station column to end_station_id
stns_vancouver = stns_vancouver.rename(columns={"start_station_id": "end_station_id"})
stns_vancouver.head()

,end_station_id
0,1
1,2
2,4
3,5
4,6


*Load rows with station_id existing in stations table*

In [ ]:
vancouver.to_sql(name='ridership', con=engine, if_exists='append', index=False, index_label='id')

***Boston***

In [ ]:
# check for missing start_station_id
missing_start = boston[-boston["start_station_id"].isin(all_stns_df["station_id"])]
missing_start

In [ ]:
# check for missing end_station_id
missing_end = boston[-boston["end_station_id"].isin(all_stns_df["station_id"])]
missing_end

In [ ]:
boston = boston[boston["start_station_id"].isin(all_stns_df["station_id"])]
boston = boston[boston["end_station_id"].isin(all_stns_df["station_id"])]
boston

In [ ]:
# get all Boston stations
stns_boston = session.query(Stations.station_id).filter(Stations.location_id==3).all()
stns_boston = pd.DataFrame(stns_boston)
stns_boston = stns_boston.rename(columns={"station_id": "start_station_id"})
stns_boston

In [ ]:
# remove Boston records with start_station_id not in stations table
df = pd.merge(boston, stns_boston, how='inner', on='start_station_id')

In [ ]:
# rename Boston station column
stns_boston = stns_boston.rename(columns={"start_station_id": "end_station_id"})
stns_boston

In [ ]:
#remove records with end_station_id not in stations table
boston_df = pd.merge(df, stns_boston, how='inner', on='end_station_id')

In [ ]:
boston_df 

In [ ]:
# load Boston records to ridership table
boston_df.to_sql(name='ridership', con=engine, if_exists='append', index=False, index_label='id')

***New York***

In [ ]:
# check for New York records with missing start_station_id in stations table
missing_start = newyork[-newyork["start_station_id"].isin(all_stns_df["station_id"])]
missing_start

In [ ]:
# check for New York records with missing end_station_id in stations table
missing_end = newyork[-newyork["end_station_id"].isin(all_stns_df["station_id"])]
missing_end

In [ ]:
newyork.count()

In [ ]:
newyork = newyork[newyork["start_station_id"].isin(all_stns_df["station_id"])]
newyork = newyork[newyork["end_station_id"].isin(all_stns_df["station_id"])]
newyork

In [ ]:
stns_newyork = session.query(Stations.station_id).filter(Stations.location_id==4).all()
stns_newyork = pd.DataFrame(stns_newyork)
stns_newyork = stns_newyork.rename(columns={"station_id": "start_station_id"})
stns_newyork

In [ ]:
ny_df = pd.merge(newyork, stns_newyork, how='inner', on='start_station_id')

In [ ]:
stns_newyork = stns_newyork.rename(columns={"start_station_id": "end_station_id"})
stns_newyork

In [ ]:
newyork_df = pd.merge(ny_df, stns_newyork, how='inner', on='end_station_id')

In [ ]:
newyork_df

In [ ]:
newyork_df.to_sql(name='ridership', con=engine, if_exists='append', index=False, index_label='id')